# Task 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for AlexNet compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Load pre-trained ResNet18 and modify the first and final layers
import torchvision.models as models

resnet18 = models.resnet18(pretrained=False)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)

# Load pre-trained AlexNet and modify the first and final layers
alexnet = models.alexnet(pretrained=False)
alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

resnet18 = resnet18.to(device)
alexnet = alexnet.to(device)
criterion = nn.CrossEntropyLoss()

# Optimizers
resnet18_optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
alexnet_optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    return model

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Train ResNet18
print("Training ResNet18...")
resnet18 = train_model(resnet18, train_loader, criterion, resnet18_optimizer)

# Train AlexNet
print("Training AlexNet...")
alexnet = train_model(alexnet, train_loader, criterion, alexnet_optimizer)

# Evaluate the models
resnet18_accuracy = evaluate_model(resnet18, test_loader)
alexnet_accuracy = evaluate_model(alexnet, test_loader)

print(f'ResNet18 Accuracy: {resnet18_accuracy:.2f}%')
print(f'AlexNet Accuracy: {alexnet_accuracy:.2f}%')


Training ResNet18...
Epoch [1/10], Loss: 0.0989


# Task 2

 Step 1: Define Models A and B

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define Model A
class ModelA(nn.Module):
    def __init__(self, activation_fn=nn.ReLU()):
        super(ModelA, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(8 * 14 * 14, 64)
        self.fc2 = nn.Linear(64, 10)
        self.activation_fn = activation_fn

    def forward(self, x):
        x = self.pool(self.activation_fn(self.conv1(x)))
        x = x.view(-1, 8 * 14 * 14)
        x = self.activation_fn(self.fc1(x))
        x = self.fc2(x)
        return x

# Define Model B
class ModelB(nn.Module):
    def __init__(self):
        super(ModelB, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(16 * 7 * 7, 64)  # Adjusted the dimensions here
        self.fc2 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 7 * 7)  # Adjusted the dimensions here
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Step 2: Prepare the MNIST Dataset

In [11]:
# Data loading and transformation
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

 Step 3: Define Training and Testing Functions

In [12]:
# Training and testing functions
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')



Step 4: Train Model A and Model B

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training Model A

print("Training Model A with ReLU")
modelA = ModelA(nn.ReLU()).to(device)
optimizerA = optim.Adam(modelA.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 11):
    train(modelA, device, train_loader, optimizerA, criterion, epoch)
    test(modelA, device, test_loader, criterion)

# Training Model B
print("\nTraining Model B with ReLU")
modelB = ModelB().to(device)
criterion = nn.CrossEntropyLoss()
optimizerB = optim.Adam(modelB.parameters(), lr=0.001)

# Training the model
for epoch in range(1, 11):
    train(modelB, device, train_loader, optimizerB, criterion, epoch)
    test(modelB, device, test_loader, criterion)


Training Model A with ReLU
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309469
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.496667
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.216169
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.155260
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.044719
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.103688
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.144899
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.216667
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.266260
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.198395

Test set: Average loss: 0.0001, Accuracy: 9674/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.096442
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.070526
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.081467
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.149162
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.116094
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.165182
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.153417
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.063115
Train Epoch: 2 [51200/60

Step 5: Train Model A with Different Activation Functions

b. Training Model A with Sigmoid

In [14]:
modelA_sigmoid = ModelA(activation_fn=nn.Sigmoid()).to(device)
optimizerA_sigmoid = optim.Adam(modelA_sigmoid.parameters(), lr=0.001)

for epoch in range(1, 11):
    train(modelA_sigmoid, device, train_loader, optimizerA_sigmoid, criterion, epoch)
    test(modelA_sigmoid, device, test_loader, criterion)



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301739
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.029783
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.010086
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.730527
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.545536
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.432978
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.317901
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.461591
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.449053
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.182138

Test set: Average loss: 0.0003, Accuracy: 9180/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.403194
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.298191
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.281279
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.265358
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.301414
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.296956
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.155827
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.322388
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.238013
T

c. Training Model A with Tanh

In [15]:
modelA_tanh = ModelA(activation_fn=nn.Tanh()).to(device)
optimizerA_tanh = optim.Adam(modelA_tanh.parameters(), lr=0.001)

for epoch in range(1, 11):
    train(modelA_tanh, device, train_loader, optimizerA_tanh, criterion, epoch)
    test(modelA_tanh, device, test_loader, criterion)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308387
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.488226
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.416735
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.204515
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.250082
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.204275
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.172798
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.274192
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.250638
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.158143

Test set: Average loss: 0.0001, Accuracy: 9580/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.124535
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.144408
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.094509
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.086327
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.087782
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.069470
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.098082
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.234843
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.080250
T

 Effect of Optimizer Learning Rate on Model B

a. Learning Rate 0.1

In [16]:
modelB_lr_0_1 = ModelB().to(device)
optimizerB_lr_0_1 = optim.Adam(modelB_lr_0_1.parameters(), lr=0.1)

for epoch in range(1, 11):
    train(modelB_lr_0_1, device, train_loader, optimizerB_lr_0_1, criterion, epoch)
    test(modelB_lr_0_1, device, test_loader, criterion)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309673
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.340111
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.309852
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.324872
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.293643
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.304275
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.312397
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.307401
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.296092
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.293898

Test set: Average loss: 0.0023, Accuracy: 1010/10000 (10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.314941
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.282449
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.282606
Train Epoch: 2 [19200/60000 (32%)]	Loss: 2.305658
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.311890
Train Epoch: 2 [32000/60000 (53%)]	Loss: 2.325495
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.319479
Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.324875
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.353167
T

b. Learning Rate 0.01

In [17]:
modelB_lr_0_01 = ModelB().to(device)
optimizerB_lr_0_01 = optim.Adam(modelB_lr_0_01.parameters(), lr=0.01)

for epoch in range(1, 11):
    train(modelB_lr_0_01, device, train_loader, optimizerB_lr_0_01, criterion, epoch)
    test(modelB_lr_0_01, device, test_loader, criterion)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320186
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.369360
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.174254
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.316907
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.012877
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.181681
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.017716
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.053824
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.180769
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.078848

Test set: Average loss: 0.0001, Accuracy: 9755/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.079012
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.071447
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.221742
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.043624
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.072584
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.019501
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.112636
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.022914
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.042568
T